In [ ]:
from transformers import BertForTokenClassification, BertModel
from data_dir import pretrained_3kmer_dir

bertForTokenClassification = BertForTokenClassification.from_pretrained(pretrained_3kmer_dir)

In [ ]:
bertModel = BertModel.from_pretrained(pretrained_3kmer_dir)

In [ ]:
import torch

label_EEE = torch.tensor('EEE')
label_EEE